In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import requests
from tqdm import tqdm 
import json

In [2]:
df = pd.read_csv('data/t20_bbb.csv')

C:\Users\anees\AppData\Local\Temp\ipykernel_13804\2925356131.py:1: DtypeWarning: Columns (51,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/t20_bbb.csv')


Removing Wrong data and anomalies

In [3]:
df=df[(df['ball']<=11)]
df=df[(df['over']<=20)]
df=df[(df['batruns']>=0) & (df['bowlruns']>=0)]
df.shape

(1675723, 57)

In [4]:
df.shape

(1675723, 57)

In [5]:
df['bowl_style'].unique()

array(['RF', 'RFM', 'LB', 'LWS', 'RMF', 'SLA', 'OB', 'LBG', 'LFM', 'LF',
       'RM', 'LMF', 'RM/OB', 'LM', 'LFM/SLA', 'OB/LB', 'LM/SLA/LWS',
       'OB/SLA', 'RAB', 'LS', '-', 'LSM', 'RSM', 'RMF/OB', 'RFM/OB',
       'OB/LBG', 'RFM/LBG', 'RM/LB', 'RM/LBG', 'LMF/RM', 'RM/RSM',
       'SLA/LWS', 'LAB', 'RMF/LB', 'RFM/LB', 'RM/OB/LB', 'RS'],
      dtype=object)

Bowlers with mixed bowling styles are grouped together

In [6]:
mixed_styles = df['bowl_style'][df['bowl_style'].str.contains('/') | df['bowl_style'].str.contains('-')].unique()
mixed_styles

array(['RM/OB', 'LFM/SLA', 'OB/LB', 'LM/SLA/LWS', 'OB/SLA', '-', 'RMF/OB',
       'RFM/OB', 'OB/LBG', 'RFM/LBG', 'RM/LB', 'RM/LBG', 'LMF/RM',
       'RM/RSM', 'SLA/LWS', 'RMF/LB', 'RFM/LB', 'RM/OB/LB'], dtype=object)

In [7]:
print("Percent of players with mixed bowling styles:",round(df[df['bowl_style'].isin(mixed_styles)].shape[0]*100/df[~df['bowl_style'].isin(mixed_styles)].shape[0],2),'%')

Percent of players with mixed bowling styles: 2.19 %


In [8]:
df.loc[df['bowl_style'].isin(mixed_styles), 'bowl_style'] = 'Mixed'
df['bowl_style'].unique()

array(['RF', 'RFM', 'LB', 'LWS', 'RMF', 'SLA', 'OB', 'LBG', 'LFM', 'LF',
       'RM', 'LMF', 'Mixed', 'LM', 'RAB', 'LS', 'LSM', 'RSM', 'LAB', 'RS'],
      dtype=object)

Renaming all bowling styles to make it more understandable

In [9]:
df['bowl_style'] = df['bowl_style'].replace(['RFM', 'RMF','RF'], 'Right Fast')
df['bowl_style'] = df['bowl_style'].replace(['LFM', 'LMF','LF'], 'Left Fast')
df['bowl_style'] = df['bowl_style'].replace(['RAB','RM','RSM','RS'], 'Right Medium')
df['bowl_style'] = df['bowl_style'].replace(['LAB','LM','LSM'], 'Left Medium')
df['bowl_style'] = df['bowl_style'].replace(['LBG', 'LB','RWS'], 'Right Wrist Spin')
df['bowl_style'] = df['bowl_style'].replace(['LWS'], 'Left Wrist Spin')
df['bowl_style'] = df['bowl_style'].replace(['OB'], 'Right Off Spin')
df['bowl_style'] = df['bowl_style'].replace(['LS', 'SLA'], 'Left Off Spin')
df['bowl_style'].unique()

array(['Right Fast', 'Right Wrist Spin', 'Left Wrist Spin',
       'Left Off Spin', 'Right Off Spin', 'Left Fast', 'Right Medium',
       'Mixed', 'Left Medium'], dtype=object)

Excluding obscure competitions

In [10]:
relevant_competitions = [
    'T20I', 'IPL', 'AUS v SL', 'T20 Blast', 'LPL', 'CPL', 'PSL', 'Big Bash League', 'BBL',
    "Men's Hundred", 'MSL', 'MSL 2018', 'Super Smash', 'National T20 Cup', 'National T20', 
    'PAK-A v NZ-A T20s', 'CSA T20 Challenge', 'CSA Provincial T20 Challenge', 
    'Syed Mushtaq Ali Trophy', 'SMA TROPHY', 'SMA TROPHY FINAL', 'ENG v IND', 'CSA T20', 
    'Provincial T20', 'Africa T20 Cup', 'PRO T20', 
    'Ireland A in Bangladesh unofficial T20I Series', 'Inter Pro T20', 'Hong Kong tour of Namibia',
    'Ireland A in Namibia T20s', 'SA20', 'PSL 2023', 'IPL 2023', 'CPL 2023', 'SL2020', 'BPL', 
    'Bangladesh Premier League', 'England tour of South Africa', 
    'Cool & Cool Presents Haier Super8 T20 Cup', 'Cool and Cool Presents Haier Mobile T-20 Cup', 
    'Scotland in United Arab Emirates T20I Match', 'Pakistan A v England Lions unofficial T20I Series', 
    'South Africa A in Zimbabwe T20Is', 'ZIM-A in NEPAL', 'DPL T20', 'ENG v NZ', 
    'Afghanistan A in Oman T20s', 'BBL 2023', 'ILT20', 'Uganda in Sri Lanka T20s', 
    'Ireland A in Nepal T20s', 'West Indies A in Nepal T20s','Afghanistan PL'
]

df=df[df['competition'].isin(relevant_competitions)]
df.shape

(1464908, 57)

Scraping player's countries through espncricinfo API

In [11]:
player_ids=df[['p_bat', 'p_bowl']].stack().drop_duplicates()
len(player_ids)
player_ids=player_ids[:10]

In [194]:
def get_country_name(player_id):
    # Format the URL with the given player_id
    url = "https://hs-consumer-api.espncricinfo.com/v1/pages/player/home?playerId={0}".format(str(player_id))
    
    # Define headers to mimic a browser request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        # Make an HTTP GET request to the URL with headers
        response = requests.get(url, headers=headers)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            
            # Safely extract the country name
            country_name = data.get('player', {}).get('country', {}).get('name', "Country name not found")
            return country_name
        else:
            return f"Failed to retrieve data from the API. Status code: {response.status_code}"
    
    except requests.exceptions.RequestException as e:
        # Catch network-related errors
        return f"Failed due to a request error: {str(e)}"
    
    except Exception as e:
        # Catch all other exceptions
        return f"An unexpected error occurred: {str(e)}"


In [195]:
# Function to save the results incrementally to JSON
def save_to_json(data, filename="data/player_country_data1.json"):
    # Open file in append mode and write the new batch of data
    with open(filename, "a") as json_file:
        json.dump(data, json_file)
        json_file.write("\n")  # Ensure each dictionary is written on a new line

# Dictionary to store player countries
player_country_dict = {}

# Process player IDs in batches of 1000
batch_size = 1000
total_batches = len(player_ids) // batch_size + 1

for i in range(total_batches):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size
    current_batch = player_ids[start_idx:end_idx]
    
    print(f"Processing batch {i+1} out of {total_batches}...")
    
    # Temporary dictionary to store results for the current batch
    batch_country_dict = {}

    for player_id in tqdm(current_batch, desc=f"Processing Players {start_idx+1} to {end_idx}"):
        country_name = get_country_name(player_id)
        batch_country_dict[player_id] = country_name
    
    # Append the current batch to the main dictionary
    player_country_dict.update(batch_country_dict)
    
    # Save current batch to JSON file
    save_to_json(batch_country_dict)

print("All batches processed and saved.")

Processing batch 1 out of 1...


Processing Players 1 to 1000: 100%|████████████████████████████████████████████████████| 10/10 [00:01<00:00,  8.91it/s]

All batches processed and saved.


## Best Batters
Computing all the batting statistics of every players

In [12]:
total_runs=df.groupby(['p_bat','bat','bat_hand']).agg({'p_match':'nunique','batruns':'sum','ballfaced':'sum'}).reset_index().sort_values(by='batruns',ascending=False)
total_runs['strike rate']=(total_runs['batruns']*100/total_runs['ballfaced']).round(2)
total_runs.rename(columns={'p_bat':'player_id','bat':'batsman','p_match':'matches_played','matches':'matches_played','batruns':'runs_scored','ballfaced':'balls_faced','strike rate':'batting_strike_rate'},inplace=True)
total_runs.head()

,player_id,batsman,bat_hand,matches_played,runs_scored,balls_faced,batting_strike_rate
1073,348144,Babar Azam,RHB,256,9812,7560,129.79
860,308967,Jos Buttler,RHB,282,9227,6376,144.71
547,232359,Colin Munro,LHB,319,8845,6274,140.98
602,249866,Alex Hales,RHB,310,8644,5784,149.45
612,253802,Virat Kohli,RHB,228,8437,6124,137.77


In [13]:
total_outs=df[df['out']==True].groupby(['p_bat','bat','bat_hand']).size().reset_index(name='count').sort_values(by='count',ascending=False)
total_outs.rename(columns={'p_bat':'player_id','bat':'batsman','count':'dismissals'},inplace=True)
total_outs.head()

,player_id,batsman,bat_hand,dismissals
580,249866,Alex Hales,RHB,297
528,232359,Colin Munro,LHB,291
893,325026,Glenn Maxwell,RHB,255
711,296597,James Vince,RHB,255
520,230559,Kieron Pollard,RHB,254


In [14]:
df1 = df[((df['batruns']==4) | (df['batruns']==6)) & (df['ballfaced']==1)].copy()
# Counting 4s and 6s in batruns
df1['4s'] = df1['batruns'].apply(lambda x: 1 if x == 4 else 0)
df1['6s'] = df1['batruns'].apply(lambda x: 1 if x == 6 else 0)
df1[['batruns', 'ballfaced', '4s', '6s']].head()

,batruns,ballfaced,4s,6s
4,4,1,1,0
11,4,1,1,0
16,4,1,1,0
50,6,1,0,1
51,6,1,0,1


In [15]:
total_boundaries=df1[((df1['batruns']==4) | (df1['batruns']==6)) & (df1['ballfaced']==1)].groupby(['p_bat','bat','bat_hand']).agg({'batruns':'sum','ballfaced':'count','4s':'sum','6s':'sum'}).reset_index()
total_boundaries.rename(columns={'p_bat':'player_id','bat':'batsman','batruns':'boundary_runs','ballfaced':'boundary_balls'},inplace=True)
total_boundaries.head()

,player_id,batsman,bat_hand,boundary_runs,boundary_balls,4s,6s
0,4068,Jim Allenby,RHB,476,109,89,20
1,4292,Travis Birt,LHB,22,5,4,1
2,4451,George Bailey,RHB,1128,252,192,60
3,4493,Dean Brownlie,RHB,420,95,75,20
4,4498,Aiden Blizzard,LHB,142,33,28,5


In [16]:
total_dots=df[(df['batruns']==0) & (df['ballfaced']==1)].groupby(['p_bat','bat','bat_hand']).agg({'ballfaced':'sum'}).reset_index()
total_dots.rename(columns={'p_bat':'player_id','bat':'batsman','ballfaced':'dots'},inplace=True)
total_dots.head()

,player_id,batsman,bat_hand,dots
0,4068,Jim Allenby,RHB,265
1,4292,Travis Birt,LHB,9
2,4451,George Bailey,RHB,562
3,4493,Dean Brownlie,RHB,158
4,4498,Aiden Blizzard,LHB,113


In [17]:
merged_df=pd.merge(total_runs,total_outs,on=['player_id','batsman','bat_hand'])
merged_df=pd.merge(merged_df,total_boundaries,on=['player_id','batsman','bat_hand'])
merged_df=pd.merge(merged_df,total_dots,on=['player_id','batsman','bat_hand'])
merged_df['dot_percent']=(merged_df['dots']*100/merged_df['balls_faced']).round(2)
merged_df['boundary_percent']=(merged_df['boundary_runs']*100/merged_df['runs_scored']).round(2)
merged_df['batting_average']=(merged_df['runs_scored']/merged_df['dismissals']).round(2)
merged_df.head()

,player_id,batsman,bat_hand,matches_played,runs_scored,balls_faced,batting_strike_rate,dismissals,boundary_runs,boundary_balls,4s,6s,dots,dot_percent,boundary_percent,batting_average
0,348144,Babar Azam,RHB,256,9812,7560,129.79,224,5228,1218,1040,178,2385,31.55,53.28,43.80
1,308967,Jos Buttler,RHB,282,9227,6376,144.71,248,5766,1244,849,395,2277,35.71,62.49,37.21
2,232359,Colin Munro,LHB,319,8845,6274,140.98,291,5690,1205,770,435,2316,36.91,64.33,30.40
3,249866,Alex Hales,RHB,310,8644,5784,149.45,297,6048,1330,966,364,2189,37.85,69.97,29.10
4,253802,Virat Kohli,RHB,228,8437,6124,137.77,189,4576,1003,721,282,1829,29.87,54.24,44.64


In [18]:
with open('data/player_country_data.json', 'r') as file:
    json_data = json.load(file)
merged_df['Country']=merged_df['player_id'].astype(str).map(json_data)
merged_df.head()

,player_id,batsman,bat_hand,matches_played,runs_scored,balls_faced,batting_strike_rate,dismissals,boundary_runs,boundary_balls,4s,6s,dots,dot_percent,boundary_percent,batting_average,Country
0,348144,Babar Azam,RHB,256,9812,7560,129.79,224,5228,1218,1040,178,2385,31.55,53.28,43.80,Pakistan
1,308967,Jos Buttler,RHB,282,9227,6376,144.71,248,5766,1244,849,395,2277,35.71,62.49,37.21,England
2,232359,Colin Munro,LHB,319,8845,6274,140.98,291,5690,1205,770,435,2316,36.91,64.33,30.40,New Zealand
3,249866,Alex Hales,RHB,310,8644,5784,149.45,297,6048,1330,966,364,2189,37.85,69.97,29.10,England
4,253802,Virat Kohli,RHB,228,8437,6124,137.77,189,4576,1003,721,282,1829,29.87,54.24,44.64,India


In [19]:
merged_df.to_csv('data/batting_stats.csv',index=False)

## Best Bowlers

In [99]:
bowl_df=df.groupby(['p_bowl','bowl','bowl_kind']).agg({'bowlruns':'sum','ballfaced':'sum','p_match':'nunique','bowl_style':'max'}).reset_index()
bowl_df['economy']=(bowl_df['bowlruns']*6/bowl_df['ballfaced']).round(2)
bowl_df.head()

,p_bowl,bowl,bowl_kind,bowlruns,ballfaced,p_match,bowl_style,economy
0,4068,Jim Allenby,pace bowler,385,277,23,Right Medium,8.34
1,4504,Ahillen Beadle,spin bowler,61,48,2,Left Off Spin,7.62
2,4508,Doug Bollinger,pace bowler,570,403,20,Left Fast,8.49
3,4527,Ryan Campbell,spin bowler,68,60,3,Right Off Spin,6.80
4,4818,Mark Cosgrove,pace bowler,19,12,1,Right Medium,9.50


In [100]:
df_dots=df[(df['bowlruns']==0) & (df['ballfaced']==1)].groupby(['p_bowl','bowl','bowl_kind']).agg({'ballfaced':'sum'}).reset_index()
df_dots.rename(columns={'ballfaced':'dotballs'},inplace=True)
df_dots.head()

,p_bowl,bowl,bowl_kind,dotballs
0,4068,Jim Allenby,pace bowler,90
1,4504,Ahillen Beadle,spin bowler,11
2,4508,Doug Bollinger,pace bowler,152
3,4527,Ryan Campbell,spin bowler,23
4,4818,Mark Cosgrove,pace bowler,2


In [101]:
bowl_outs=df[df['out']==True].groupby(['p_bowl','bowl','bowl_kind']).size().reset_index(name='outs')
merged_df=pd.merge(bowl_df,bowl_outs,on=['p_bowl','bowl','bowl_kind'])
merged_df=pd.merge(merged_df,df_dots,on=['p_bowl','bowl','bowl_kind'])
merged_df['dotballs_percent']=(merged_df['dotballs']*100/merged_df['ballfaced']).round(2)
merged_df['bowling_average']=(merged_df['bowlruns']/merged_df['outs']).round(2)
merged_df['bowling_strike_rate']=(merged_df['ballfaced']/merged_df['outs']).round(2)
merged_df=merged_df.sort_values(by='bowling_average', ascending=True)
merged_df.rename(columns={'p_bowl':'player_id','bowl':'bowler','bowlruns':'runs_conceded','ballfaced':'ball_bowled','p_match':'matches_played','bowl_style':'bowling_style','outs':'dismissals'},inplace=True)
merged_df.head()

,player_id,bowler,bowl_kind,runs_conceded,ball_bowled,matches_played,bowling_style,economy,dismissals,dotballs,dotballs_percent,bowling_average,bowling_strike_rate
964,451947,Milind Kumar,spin bowler,0,1,1,Right Off Spin,0.0,1,1,100.0,0.0,1.0
1135,520084,Spyridon Goustis,pace bowler,0,6,1,Right Medium,0.0,1,6,100.0,0.0,6.0
3346,1344843,Kashif Siddique,mixture/unknown,1,2,1,Mixed,3.0,1,1,50.0,1.0,2.0
1757,794713,Kushal Bhurtel,pace bowler,1,6,1,Right Medium,1.0,1,6,100.0,1.0,6.0
373,248920,Devon Thomas,pace bowler,4,6,1,Right Medium,4.0,2,3,50.0,2.0,3.0


In [102]:
merged_df['Country']=merged_df['player_id'].astype(str).map(json_data)
merged_df.head()

,player_id,bowler,bowl_kind,runs_conceded,ball_bowled,matches_played,bowling_style,economy,dismissals,dotballs,dotballs_percent,bowling_average,bowling_strike_rate,Country
964,451947,Milind Kumar,spin bowler,0,1,1,Right Off Spin,0.0,1,1,100.0,0.0,1.0,United States of America
1135,520084,Spyridon Goustis,pace bowler,0,6,1,Right Medium,0.0,1,6,100.0,0.0,6.0,Greece
3346,1344843,Kashif Siddique,mixture/unknown,1,2,1,Mixed,3.0,1,1,50.0,1.0,2.0,Saudi Arabia
1757,794713,Kushal Bhurtel,pace bowler,1,6,1,Right Medium,1.0,1,6,100.0,1.0,6.0,Nepal
373,248920,Devon Thomas,pace bowler,4,6,1,Right Medium,4.0,2,3,50.0,2.0,3.0,West Indies


In [103]:
merged_df.to_csv('data/bowling_stats.csv',index=False)

# Batter's performaces against every bowling type

In [25]:
total_runs=df.groupby(['p_bat','bat','bat_hand','bowl_style']).agg({'p_match':'nunique','batruns':'sum','ballfaced':'sum'}).reset_index().sort_values(by='batruns',ascending=False)
total_runs['strike rate']=(total_runs['batruns']*100/total_runs['ballfaced']).round(2)
# total_runs.rename(columns={'p_bat':'player_id','bat':'batsman','batruns':'runs_scored','ballfaced':'balls_faced',
#                            'p_match':'matches_played','strike rate':'batting_strike_rate'},inplace=True)
total_runs.head()

,p_bat,bat,bat_hand,bowl_style,p_match,batruns,ballfaced,strike rate
6700,348144,Babar Azam,RHB,Right Fast,235,3906,2889,135.20
3763,249866,Alex Hales,RHB,Right Fast,261,3787,2460,153.94
5362,308967,Jos Buttler,RHB,Right Fast,237,3743,2437,153.59
95,5334,Aaron Finch,RHB,Right Fast,215,3591,2488,144.33
3400,232359,Colin Munro,LHB,Right Fast,257,3498,2428,144.07


In [26]:
total_outs=df[df['out']==True].groupby(['p_bat','bat','bat_hand','bowl_style']).size().reset_index(name='count').sort_values(by='count',ascending=False)
total_outs.rename(columns={'count':'dismissals'},inplace=True)
total_outs.head()

,p_bat,bat,bat_hand,bowl_style,dismissals
2653,249866,Alex Hales,RHB,Right Fast,121
2349,230559,Kieron Pollard,RHB,Right Fast,114
3367,298438,Jason Roy,RHB,Right Fast,113
3250,296597,James Vince,RHB,Right Fast,110
2385,232359,Colin Munro,LHB,Right Fast,108


In [27]:
merged_df=pd.merge(total_runs,total_outs,on=['p_bat','bat','bat_hand','bowl_style'])
merged_df.columns

Index(['p_bat', 'bat', 'bat_hand', 'bowl_style', 'p_match', 'batruns',
       'ballfaced', 'strike rate', 'dismissals'],
      dtype='object')

In [28]:
merged_df['average']=(merged_df['batruns']/merged_df['dismissals']).round(2)
merged_df=merged_df.sort_values(by='average',ascending=False)
merged_df.head()

,p_bat,bat,bat_hand,bowl_style,p_match,batruns,ballfaced,strike rate,dismissals,average
1535,671805,George Munsey,LHB,Right Medium,18,262,159,164.78,1,262.0
1828,311158,Ben Stokes,LHB,Left Off Spin,24,225,137,164.23,1,225.0
2046,1163154,Sami Sohail,RHB,Right Fast,7,202,148,136.49,1,202.0
2108,47015,Jacques Rudolph,LHB,Right Medium,20,197,165,119.39,1,197.0
2221,1151288,Sai Sudharsan,LHB,Left Off Spin,19,185,119,155.46,1,185.0


In [29]:
merged_df['Country']=merged_df['p_bat'].astype(str).map(json_data)
merged_df.head()

,p_bat,bat,bat_hand,bowl_style,p_match,batruns,ballfaced,strike rate,dismissals,average,Country
1535,671805,George Munsey,LHB,Right Medium,18,262,159,164.78,1,262.0,Scotland
1828,311158,Ben Stokes,LHB,Left Off Spin,24,225,137,164.23,1,225.0,England
2046,1163154,Sami Sohail,RHB,Right Fast,7,202,148,136.49,1,202.0,Malawi
2108,47015,Jacques Rudolph,LHB,Right Medium,20,197,165,119.39,1,197.0,South Africa
2221,1151288,Sai Sudharsan,LHB,Left Off Spin,19,185,119,155.46,1,185.0,India


In [30]:
merged_df.rename(columns={'p_bat':'player_id',
                          'bat':'batsman',
                          'bowl_style':'bowling_style',
                          'p_match':'matches_played',
                          'batruns':'runs_scored',
                          'ballfaced':'balls_faced',
                          'strike rate':'batting_strike_rate',
                          'average':'batting_average'},inplace=True)
merged_df.head()

,player_id,batsman,bat_hand,bowling_style,matches_played,runs_scored,balls_faced,batting_strike_rate,dismissals,batting_average,Country
1535,671805,George Munsey,LHB,Right Medium,18,262,159,164.78,1,262.0,Scotland
1828,311158,Ben Stokes,LHB,Left Off Spin,24,225,137,164.23,1,225.0,England
2046,1163154,Sami Sohail,RHB,Right Fast,7,202,148,136.49,1,202.0,Malawi
2108,47015,Jacques Rudolph,LHB,Right Medium,20,197,165,119.39,1,197.0,South Africa
2221,1151288,Sai Sudharsan,LHB,Left Off Spin,19,185,119,155.46,1,185.0,India


In [31]:
merged_df.to_csv('data/batters_against_bowlingtype.csv', index=False)

# Bowler's performance against right and left handed batters

In [32]:
bowl_df=df.groupby(['p_bowl','bowl','bat_hand','bowl_kind']).agg({'bowlruns':'sum','ballfaced':'sum','p_match':'nunique','bowl_style':'max'}).reset_index()
bowl_df['economy']=(bowl_df['bowlruns']*6/bowl_df['ballfaced']).round(2)
bowl_df.head()

,p_bowl,bowl,bat_hand,bowl_kind,bowlruns,ballfaced,p_match,bowl_style,economy
0,4068,Jim Allenby,LHB,pace bowler,106,82,13,Right Medium,7.76
1,4068,Jim Allenby,RHB,pace bowler,279,195,21,Right Medium,8.58
2,4504,Ahillen Beadle,RHB,spin bowler,61,48,2,Left Off Spin,7.62
3,4508,Doug Bollinger,LHB,pace bowler,180,130,18,Left Fast,8.31
4,4508,Doug Bollinger,RHB,pace bowler,390,273,20,Left Fast,8.57


In [33]:
bowl_outs=df[df['out']==True].groupby(['p_bowl','bowl','bat_hand','bowl_kind']).size().reset_index(name='outs')
bowl_outs.head()

,p_bowl,bowl,bat_hand,bowl_kind,outs
0,4068,Jim Allenby,LHB,pace bowler,1
1,4068,Jim Allenby,RHB,pace bowler,7
2,4504,Ahillen Beadle,RHB,spin bowler,1
3,4508,Doug Bollinger,LHB,pace bowler,6
4,4508,Doug Bollinger,RHB,pace bowler,13


In [34]:
merged_df=pd.merge(bowl_df,bowl_outs,on=['p_bowl','bowl','bat_hand','bowl_kind'])
merged_df['average']=(merged_df['bowlruns']/merged_df['outs']).round(2)
merged_df['strike rate']=(merged_df['ballfaced']/merged_df['outs']).round(2)
merged_df=merged_df[(merged_df['p_match']>25) & (merged_df['ballfaced']>500)].sort_values(by='average', ascending=True)
merged_df.head()

,p_bowl,bowl,bat_hand,bowl_kind,bowlruns,ballfaced,p_match,bowl_style,economy,outs,average,strike rate
4799,1178061,Alpesh Ramjani,RHB,spin bowler,474,632,34,Left Off Spin,4.50,60,7.90,10.53
4805,1178080,Vraj Patel,RHB,spin bowler,464,556,33,Left Off Spin,5.01,47,9.87,11.83
5178,1218731,Elam Bharathi,RHB,spin bowler,509,591,36,Left Off Spin,5.17,42,12.12,14.07
4793,1178055,Yalinde Nkanya,RHB,spin bowler,697,782,45,Left Off Spin,5.35,56,12.45,13.96
170,26139,Frank Nsubuga,RHB,spin bowler,569,716,46,Left Off Spin,4.77,45,12.64,15.91


In [35]:
merged_df['Country']=merged_df['p_bowl'].astype(str).map(json_data)
merged_df.head()

,p_bowl,bowl,bat_hand,bowl_kind,bowlruns,ballfaced,p_match,bowl_style,economy,outs,average,strike rate,Country
4799,1178061,Alpesh Ramjani,RHB,spin bowler,474,632,34,Left Off Spin,4.50,60,7.90,10.53,Uganda
4805,1178080,Vraj Patel,RHB,spin bowler,464,556,33,Left Off Spin,5.01,47,9.87,11.83,Kenya
5178,1218731,Elam Bharathi,RHB,spin bowler,509,591,36,Left Off Spin,5.17,42,12.12,14.07,Germany
4793,1178055,Yalinde Nkanya,RHB,spin bowler,697,782,45,Left Off Spin,5.35,56,12.45,13.96,Tanzania
170,26139,Frank Nsubuga,RHB,spin bowler,569,716,46,Left Off Spin,4.77,45,12.64,15.91,Uganda


In [36]:
merged_df.rename(columns={'p_bowl':'player_id',
                          'bowl':'bowler',
                          'bowl_style':'bowling_style',
                          'bowlruns':'runs_conceded',
                          'batruns':'runs_scored',
                          'ballfaced':'ball_bowled',
                          'p_match':'matches_played',
                          'outs':'dismissals',
                          'average':'bowling_average',
                          'strike rate':'bowling_strike_rate'},inplace=True)
merged_df.columns

Index(['player_id', 'bowler', 'bat_hand', 'bowl_kind', 'runs_conceded',
       'ball_bowled', 'matches_played', 'bowling_style', 'economy',
       'dismissals', 'bowling_average', 'bowling_strike_rate', 'Country'],
      dtype='object')

In [37]:
merged_df.to_csv('data/Bowling_against_left_right_handers.csv',index=False)

## Overall Statistics by grounds

In [95]:
df_ground=df.groupby(['ground']).agg({'p_match':'nunique','batruns':'sum','ballfaced':'sum','country':'max'}).reset_index()
df_ground['avg_score']=round(df_ground['batruns']*120/df_ground['ballfaced'],2)
df_ground.drop(columns=['batruns','ballfaced'],inplace=True)
df_ground.rename(columns={'p_match':'matches_played'},inplace=True)
df_ground.head()

,ground,matches_played,country,avg_score
0,"Abhimanyu Cricket Academy, Dehradun",10,India,135.59
1,"Achimota Senior Secondary School A Field, Accra",6,Ghana,127.03
2,"Achimota Senior Secondary School B Field, Accra",6,Ghana,117.40
3,Adelaide Oval,68,Australia,159.26
4,"Aigburth, Liverpool",2,England,136.00


## Overall Ground Statistics by Innings

In [96]:
df_ground_inns=df.groupby(['inns','ground']).agg({'batruns':'sum','ballfaced':'sum'}).reset_index()
df_ground_inns['inns_avg_score']=round(df_ground_inns['batruns']*120/df_ground_inns['ballfaced'],2)
df_ground_inns.drop(columns=['batruns','ballfaced'],inplace=True)
df_ground_inns.head()

,inns,ground,inns_avg_score
0,1,"Abhimanyu Cricket Academy, Dehradun",142.35
1,1,"Achimota Senior Secondary School A Field, Accra",151.07
2,1,"Achimota Senior Secondary School B Field, Accra",124.64
3,1,Adelaide Oval,163.22
4,1,"Aigburth, Liverpool",128.96


In [97]:
merged_df=pd.merge(df_ground,df_ground_inns[df_ground_inns['inns']==1],on=['ground']).drop(columns='inns').rename(columns={'inns_avg_score':'1st_inns_avg_score'})
merged_df=pd.merge(merged_df,df_ground_inns[df_ground_inns['inns']==2],on=['ground']).drop(columns='inns').rename(columns={'inns_avg_score':'2nd_inns_avg_score'})
merged_df.sort_values(by='matches_played',ascending=False,inplace=True)
merged_df.head()

,ground,matches_played,country,avg_score,1st_inns_avg_score,2nd_inns_avg_score
200,"Shere Bangla National Stadium, Mirpur",260,Bangladesh,140.53,142.71,138.19
61,Dubai International Cricket Stadium,177,United Arab Emirates,146.14,147.83,144.31
180,Rawalpindi Cricket Stadium,134,Pakistan,163.01,163.07,162.95
196,Sharjah Cricket Stadium,121,United Arab Emirates,148.14,149.24,146.93
247,"Wankhede Stadium, Mumbai",101,India,166.21,165.69,166.76


In [98]:
merged_df.to_csv("data/Overall_Ground_Stats.csv")